# Mutual information estimates

In [1]:
# Custom code imports
from generate_time_series import (load_two_body_problem_time_series,
                                  load_belousov_zhabotinsky_time_series,
                                  load_lorenz_attractor_time_series)

from datasets import (chop_time_series_into_chunks,
                      split_chunks_into_windows_and_targets)

In [2]:
# Standard code imports
import numpy as np
import functools 

from typing import Tuple, Optional
import numpy.typing
NDArray = numpy.typing.NDArray[np.floating]

In [3]:
# Helpers
def time_series_2_windows_and_targets(time_series: NDArray,
                                      window_len: int = 10,
                                      target_len: int = 1,
                                      reverse: bool = False) -> Tuple[NDArray, NDArray]:
    chunks = chop_time_series_into_chunks(time_series,
                                          chunk_len=window_len+target_len,
                                          reverse=reverse,
                                          take_each_nth_chunk=3)
    windows, targets = split_chunks_into_windows_and_targets(chunks, target_len=target_len)
    return windows, targets


def flatten_last_dim(array: NDArray) -> NDArray:
    assert array.ndim >= 2
    return array.reshape((*array.shape[:-2], -1))

## `sklearn.feature_selection.mutual_info_regression`

In [4]:
# # This function only works with discrete inputs (handy for categorization/clusterization).
# # It is unusable for the float (continuous) vectors we are dealing with here.
# from sklearn.metrics import mutual_info_score

In [5]:
# See https://www.blog.trainindata.com/mutual-information-with-python/
from sklearn.feature_selection import mutual_info_regression

In [6]:
def calculate_mutual_info_for_dataset(ts: NDArray, dim: int = 0) -> Tuple[NDArray, NDArray]:
    assert 0 <= dim < ts.shape[1]

    forward_windows, forward_targets = time_series_2_windows_and_targets(ts)
    backward_windows, backward_targets = time_series_2_windows_and_targets(ts, reverse=True)

    # Each window or target is two-dimensional. I extract just one dimension
    backward_windows = backward_windows[:, :, dim]
    forward_windows = forward_windows[:, :, dim]
    # ... and assume target_len=1, so take the 0-th point in target.
    forward_targets = forward_targets[:, 0, dim]
    backward_targets = backward_targets[:, 0, dim]
    # Note: `mutual_info_regression` only accepts 1-dimensional y's.
    # So I'm forced to pick only one dimension from targets, although I could
    # flatten the windows instead of extracting one dimension from it.

    return (mutual_info_regression(forward_windows, forward_targets),
            mutual_info_regression(backward_windows, backward_targets))

In [7]:
def print_mutual_info(ts: NDArray, comment: str) -> None:
    forward, backward = calculate_mutual_info_for_dataset(ts)
    print(comment, "forward", forward)
    print(comment, "backward", backward)

In [8]:
print_mutual_info(load_two_body_problem_time_series(), "kepler")
print()
print_mutual_info(load_belousov_zhabotinsky_time_series(), "belousov_zhabotinsky")
print()
print_mutual_info(load_lorenz_attractor_time_series(), "lorenz")

kepler forward [4.1544507  4.1679777  4.19014217 4.23306892 4.40380346 4.52204556
 4.6513439  4.83707335 5.07009635 5.38225062]
kepler backward [4.1544507  4.16829816 4.19438907 4.23299843 4.40411548 4.52240252
 4.65179145 4.83874348 5.06217402 5.38334711]

belousov_zhabotinsky forward [5.10297078 5.11591333 5.12894595 5.15271997 5.25902738 5.40578542
 5.4406048  5.4858174  5.54904934 5.63826422]
belousov_zhabotinsky backward [5.10354369 5.11591333 5.12937203 5.1472664  5.22999303 5.39968754
 5.42965822 5.47702134 5.53905855 5.6281999 ]

lorenz forward [1.10491599 1.18068127 1.28136143 1.39081549 1.52221169 1.68715116
 1.89150051 2.17183271 2.57736067 3.26623382]
lorenz backward [1.10105144 1.18418257 1.28136143 1.38803675 1.51959223 1.68800725
 1.90041139 2.17751784 2.57858673 3.25964205]


The numbers are about the same, within reasonable accuracy, for `forward` and `backward`.
This is not what we expect.

## gregversteeg/NPEET

In [9]:
# Install the module from GitHub
!git clone https://github.com/gregversteeg/NPEET.git
#!rm -rf NPEET

Cloning into 'NPEET'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 129 (delta 21), reused 35 (delta 19), pack-reused 87
Receiving objects: 100% (129/129), 317.14 KiB | 3.82 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [10]:
# The module's suggested installation method doesn't work,
# so we just find the right source file in the directory tree.
from NPEET.npeet import entropy_estimators

In [11]:
def calculate_mutual_info_for_dataset(ts: NDArray,
                                      window_len: int,
                                      target_len: int,
                                      project_on_coordinate: Optional[int] = None) -> Tuple[float, float]:
    forward_windows, forward_targets = time_series_2_windows_and_targets(ts, window_len=window_len,
                                                                         target_len=target_len)
    backward_windows, backward_targets = time_series_2_windows_and_targets(ts, window_len=window_len,
                                                                           target_len=target_len, reverse=True)
    
    if project_on_coordinate is None:
        # Each window or target is two-dimensional, so I flatten them.
        backward_windows = flatten_last_dim(backward_windows)
        forward_windows = flatten_last_dim(forward_windows)
        backward_targets = flatten_last_dim(backward_targets)
        forward_targets = flatten_last_dim(forward_targets)
    else:
        assert project_on_coordinate < ts.shape[1], "Index too large"
        
        backward_windows = backward_windows[:, :, project_on_coordinate]
        forward_windows = forward_windows[:, :, project_on_coordinate]
        backward_targets = backward_targets[:, :, project_on_coordinate]
        forward_targets = forward_targets[:, :, project_on_coordinate]
    
    return (entropy_estimators.mi(forward_windows, forward_targets),
            entropy_estimators.mi(backward_windows, backward_targets))

def apply_mi_over_len_grid(ts: NDArray, **kwargs) -> None:
    print("window+target: (backward - forward) / forward")
    for window_len in [3, 5, 10, 15]:
        for target_len in [1, 3, 5]:
            forward, backward = calculate_mutual_info_for_dataset(ts=ts,
                                                                  window_len=window_len,
                                                                  target_len=target_len,
                                                                  **kwargs)
            # Print relative difference between backward and forward.
            print(f"{window_len}+{target_len}:\t{(backward - forward) / forward:.2e}")

In [12]:
apply_mi_over_len_grid(load_two_body_problem_time_series())

window+target: (backward - forward) / forward
3+1:	-5.47e-05
3+3:	2.16e-16
3+5:	-5.48e-05
5+1:	0.00e+00
5+3:	5.48e-05
5+5:	5.49e-05
10+1:	-3.26e-03
10+3:	-3.28e-03
10+5:	-3.23e-03
15+1:	-3.11e-03
15+3:	-3.12e-03
15+5:	-3.10e-03


In [13]:
apply_mi_over_len_grid(load_lorenz_attractor_time_series())

window+target: (backward - forward) / forward
3+1:	-2.43e-04
3+3:	-5.12e-04
3+5:	1.61e-04
5+1:	-6.92e-04
5+3:	-9.65e-04
5+5:	0.00e+00
10+1:	-1.17e-03
10+3:	-1.98e-03
10+5:	-1.43e-03
15+1:	-4.15e-03
15+3:	-4.15e-03
15+5:	-3.57e-03


In [14]:
apply_mi_over_len_grid(load_belousov_zhabotinsky_time_series())

window+target: (backward - forward) / forward
3+1:	7.46e-04
3+3:	3.03e-04
3+5:	-2.84e-05
5+1:	7.33e-04
5+3:	8.27e-04
5+5:	-1.51e-04
10+1:	1.71e-02
10+3:	1.68e-02
10+5:	1.57e-02
15+1:	1.69e-02
15+3:	1.73e-02
15+5:	1.75e-02


In [15]:
# Let's try to apply it to random noise
apply_mi_over_len_grid(np.random.normal(loc=1, scale=1, size=(4000, 3)))

window+target: (backward - forward) / forward
3+1:	-1.20e+00
3+3:	-1.19e+00
3+5:	-1.27e-01
5+1:	1.62e-01
5+3:	-6.96e-01
5+5:	-3.52e-03
10+1:	3.36e+00
10+3:	-7.63e-01
10+5:	-4.18e-01
15+1:	4.85e-02
15+3:	-3.85e-02
15+5:	8.68e-01


### So far, nothing interpretable as a trend. Let's try projecting to have only one coordinate instead of flattening, which could have destroyed the distribution-based magic.

In [16]:
apply_mi_over_len_grid(load_two_body_problem_time_series(), project_on_coordinate=0)

window+target: (backward - forward) / forward
3+1:	1.92e-04
3+3:	1.00e-03
3+5:	-4.05e-03
5+1:	1.08e-02
5+3:	4.06e-03
5+5:	1.37e-03
10+1:	-1.08e-02
10+3:	-1.60e-02
10+5:	-2.00e-02
15+1:	-1.13e-02
15+3:	-1.73e-02
15+5:	-2.04e-02


In [17]:
apply_mi_over_len_grid(load_two_body_problem_time_series(), project_on_coordinate=1)

window+target: (backward - forward) / forward
3+1:	2.47e-03
3+3:	-9.48e-04
3+5:	3.98e-03
5+1:	-9.91e-03
5+3:	-3.96e-03
5+5:	-5.70e-04
10+1:	4.21e-03
10+3:	6.60e-03
10+5:	1.06e-02
15+1:	4.02e-03
15+3:	6.96e-03
15+5:	1.05e-02


In [18]:
apply_mi_over_len_grid(load_lorenz_attractor_time_series(), project_on_coordinate=0)

window+target: (backward - forward) / forward
3+1:	2.16e-03
3+3:	4.31e-03
3+5:	-2.30e-03
5+1:	-2.33e-05
5+3:	-3.80e-04
5+5:	0.00e+00
10+1:	1.38e-03
10+3:	1.84e-03
10+5:	-6.66e-04
15+1:	3.19e-03
15+3:	1.34e-04
15+5:	4.99e-04


In [19]:
apply_mi_over_len_grid(load_lorenz_attractor_time_series(), project_on_coordinate=1)

window+target: (backward - forward) / forward
3+1:	-4.00e-03
3+3:	-3.44e-03
3+5:	2.24e-04
5+1:	-4.72e-03
5+3:	1.72e-03
5+5:	-1.14e-05
10+1:	-1.86e-03
10+3:	3.18e-04
10+5:	-1.67e-03
15+1:	2.74e-03
15+3:	-2.42e-04
15+5:	-1.30e-03


In [20]:
apply_mi_over_len_grid(load_lorenz_attractor_time_series(), project_on_coordinate=2)

window+target: (backward - forward) / forward
3+1:	5.41e-04
3+3:	1.44e-03
3+5:	3.51e-03
5+1:	-1.49e-03
5+3:	-1.08e-03
5+5:	0.00e+00
10+1:	-3.79e-03
10+3:	-3.50e-03
10+5:	-2.86e-03
15+1:	-7.97e-03
15+3:	-3.88e-03
15+5:	-2.65e-03


In [21]:
apply_mi_over_len_grid(load_belousov_zhabotinsky_time_series(), project_on_coordinate=0)

window+target: (backward - forward) / forward
3+1:	2.58e-03
3+3:	-5.06e-05
3+5:	-6.88e-04
5+1:	3.43e-03
5+3:	9.75e-04
5+5:	1.05e-04
10+1:	-2.43e-02
10+3:	-2.94e-02
10+5:	-3.15e-02
15+1:	-2.46e-02
15+3:	-2.95e-02
15+5:	-3.13e-02


In [22]:
apply_mi_over_len_grid(load_belousov_zhabotinsky_time_series(), project_on_coordinate=1)

window+target: (backward - forward) / forward
3+1:	3.67e-04
3+3:	2.15e-04
3+5:	-5.92e-04
5+1:	1.49e-03
5+3:	1.06e-03
5+5:	3.47e-04
10+1:	1.73e-02
10+3:	1.63e-02
10+5:	1.37e-02
15+1:	1.78e-02
15+3:	1.76e-02
15+5:	1.61e-02


In [23]:
apply_mi_over_len_grid(load_belousov_zhabotinsky_time_series(), project_on_coordinate=2)

window+target: (backward - forward) / forward
3+1:	-5.07e-03
3+3:	1.00e-04
3+5:	2.89e-03
5+1:	-8.53e-03
5+3:	-3.92e-03
5+5:	-3.39e-04
10+1:	1.36e-02
10+3:	2.63e-02
10+5:	3.16e-02
15+1:	1.24e-02
15+3:	2.30e-02
15+5:	2.82e-02
